In [2]:
!pip install PyPDF2

  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   -------------------------- ------------- 153.6/232.6 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [1]:
!pip install transformers

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import warnings
warnings.filterwarnings("ignore") # Ignoring the warnings in the notebook

In [3]:
# Initialize the model and tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
def paraphrase_text(text, format_choice='paragraph'):
    # Check if the input contains bullet points (lines starting with numbers or dots)
    if "1." in text:
        # Split by newlines to isolate bullet points
        bullet_points = text.split("\n")
        paraphrased_points = []
        
        # Paraphrase each bullet point
        for point in bullet_points:
            if point.strip():  # Skip empty lines
                input_text = f"paraphrase: {point.strip()} </s>"
                input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
                outputs = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
                paraphrased_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                paraphrased_points.append(paraphrased_text)
        
        # Return paraphrased bullet points or paragraph format
        if format_choice == 'bullet_points':
            return paraphrased_points  # Return list of paraphrased bullet points
        else:
            return " ".join(paraphrased_points)  # Join paraphrased points into a paragraph
    else:
        # If no bullet points, paraphrase the whole text as a paragraph
        input_text = f"paraphrase: {text.strip()} </s>"
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
        paraphrased_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return paraphrased_text

In [5]:
# Input paragraph
scope_paragraph = """
The minimum specified scope of work to be undertaken by the bidder for setting up and operating ISWM Project is mentioned below. The Selected Bidder shall develop, deploy, integrate and support the required deliverables as per the scope and schedule of the contract along with the installation of hardware as detailed out in the further sections.
As a part of the technical proposal bidder is expected to submit the proposed systems complete technology stack & architecture.
After signing of the Agreement, the Systems Integrator needs to deploy the team proposed within 15 days of receiving the work order for the Project and ensure that a Project Inception Report is submitted to PMC which should cover following aspects:
1. Names of the Project Team members, their roles & responsibilities
2. Approach & methodology to be adopted to implement the Project (which should be in line with what has been proposed during bidding stage but may have value additions / learning in the interest of the project).
3. Responsibility matrix for all stakeholders
4. Risks the bidder anticipates and the plans they have towards their mitigation.
5. Detailed Project Plan, specifying dependencies between various project activities/sub-activities and their timelines.
"""

In [6]:
# User input for format preference
format_choice = input("Enter the format you want ('bullet_points' or 'paragraph'): ").strip()

Enter the format you want ('bullet_points' or 'paragraph'): paragraph


In [7]:
# Paraphrase and display output
paraphrased_output = paraphrase_text(scope_paragraph, format_choice)

In [8]:
if format_choice == 'bullet_points':
    print("\nParaphrased Bullet Points:")
    for i, bullet in enumerate(paraphrased_output, 1):
        print(f"{i}. {bullet}")
else:
    print("\nParaphrased Paragraph:")
    print(paraphrased_output)


Paraphrased Paragraph:
True Paraphrase: As part of the technical proposal bidder is expected to submit the proposed systems complete technology stack & architecture. True True False 3. Responsibility matrix for all stakeholders for all stakeholders for all stakeholders. True True


# Have not been able to set the tone and get contextual paraphrasing in this version as well and the T5 transformer has performed badly even for a generic rewording paraphrasing output.